## 랜덤 포레스트
- 결정트리를 랜덤하게 만들어 결정 트리(나무)숲을 만듬, 결정 트리의 예측을 사용해 최종 예측을 만듬
- 부트스트랩 방식으로 샘플링하여 분류한 데이터
- 각 노드 분할 시 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할 찾음
- RandomFrorestClassifier는 전체 특성 개수의 제곱근만큼의 특성 선택
- RandomFrorestRegressor는 전체 특성 사용
- 랜덤하게 선택한 샘플과 특성을 사용하여 훈련 세트에 과대적합 되는 것을 막아줌

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# 데이터 불러오기
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
# numpy 형태로 변환
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine[['class']].to_numpy()

In [4]:
# 훈련/테스트 셋으로 나누기
train_input, test_input, train_target, test_target = train_test_split(
data, target, test_size=0.2, random_state = 42)

In [ ]:
# RandomForestClassifier 기본적으로 100개의 결정 트리 사용
# 모든 cpu 사용 권장 -> n_jobs=-1
# return_train_score 매개변수를 True로 지정, 훈련 세트 점수도 반환
# 훈련 세트와 검증 세트의 점수를 비교하면 과대적합을 파악하는데 용이

In [8]:
# 교차 검증
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

In [9]:
scores = cross_validate(rf, train_input, train_target,
                       return_train_score=True, n_jobs=-1)

In [10]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 훈련 세트, 검증세트 점수 동시 반환

0.9973541965122431 0.8905151032797809


In [ ]:
# 훈련 세트에 과대적합, 이 예제는 매우 간단하고 특성이 많지 않아 그리드 서치를 사용해도 하이퍼파라미터 튜닝 결과가 크게 나아지지 않음
# 랜덤포레스트는 트리의 앙상블로 결정트리가 제공하는 주요 매개변수 모두 제공
# 랜덤포레스트는 각 결정트리의 특성 중요도를 취합한 것

In [11]:
rf.fit(train_input, train_target)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [12]:
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [ ]:
# 결정트리의 와인 분류 특성 중요도와 비교했을 때 당도의 중요도가 감소, 알콜과 pH 중요도 상승
# 랜덤포레스트는 특성 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문, 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여
# 과대적합을 줄이고 일반화 성능을 높이는 데 도움됨

In [13]:
# 부트스트랩 포함되지 않고 남은 OOB 샘플로 모델 평가(검증 세트 역할)
# 각 결정트리의 OOB 점수를 평균하여 출력
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
rf.oob_score_

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.8934000384837406

In [ ]:
# 교차검증 점수와 비슷하다

## 엑스트라 트리
- 랜덤포레스트와 비슷하게 100개의 결정트리 훈련, 동일한 매개변수 지원, 전체 특성 중 일부 특성을 랜덤하게 선택해서 노트 분할에 사용
- 차이점은 부트스트랩 샘플을 사용하지 않음
- 결정트리 만들 때 전체 훈련 세트 사용
- 노드 분할 시 가장 좋은 분할이 아닌 무작위 분할(결정트리의 splitter='random')
- 무작위 분할은 하나의 트리에서는 성능이 낮았지만 많은 트리를 앙상블하기 때문에 과대적합을 막고 검증 세트 점수 높임

In [14]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                       return_train_score=True, n_jobs=-1)

In [15]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
# 랜덤 포레스트와 비슷한 결과, 보통 엑스트라 트리가 무작위성이 더 크기 때문에 더 많은 결정트리를 훈련, 랜덤하게 노드를 분할하여 빠른 계산속도가 장점

In [16]:
# 특성 중요도 제공
et.fit(train_input, train_target)
print(et.feature_importances_)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


[0.20183568 0.52242907 0.27573525]


In [17]:
# 결정트리보다 당도에 대한 의존성이 낮음

## 그레이디언트 부스팅
- 깊이가 얕은 결정 트리 사용하여 이전 트리 오차 보완하는 방식의 앙상블
- 기본적으로 깊이가 3인 결정트리 100개 사용
- 과대적합에 강하고 일반적으로 높은 일반화 성능 기대
- 경사 하강법 사용하여 트리를 앙상블에 추가
- 분류에서는 로지스틱 손실함수를 사용, 회귀에서는 평균 제곱 오차 함수 사용
- 경사하강법은 손실함수를 산으로 정의하고 가장 낮은 곳을 찾아 내려오는 과정
- 가장 낮은 곳을 찾아 내려올 때 모델의 가중치와 절편을 조금씩 바꿈
- 그래디언트 부스팅은 결정 트리를 계속 추가하면서 가장 낮은 곳 찾아 이동

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, 
                       return_train_score=True, n_jobs=-1)

In [19]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
# 과대적합 되지 않음, 결정트리 개수를 늘려도 과대적합에 강함, 학습률 증가하고, 트리의 개수를 늘리면 성능 향상

In [21]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs=-1)
# 결정트리개수를 500개로 5배나 늘림, learning_rate=0.1에서 0.2로 늘림

In [22]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [23]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.15872278 0.68010884 0.16116839]


In [24]:
# 그레디언트 부스팅이 랜덤포레스트보다 당도에 더 집중
# 훈련에 사용할 훈련 세트 비율은 subsample 기본값 1.0으로 전체 사용, 1보다 작으면 일부 사용
# 순서대로 트리를 추가하기 때문에 훈련속도가 느림, n_jobs 매개변수가 없음

## 히스토그램 기반 그레이디언트 부스팅
- 입력 특성을 256개의 구간으로 나눔
- 노드 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음
- 256개의 구간 중에서 하나를 떼어놓고 누락된 값을 위해서 사용(입력에 누락된 특성이 있더라도 따로 전처리할 필요x)
- n_estimators대신에 부스팅 반복 횟수를 지정하는 max_iter를 사용 (성능을 높이려면 테스트)

In [25]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)

In [26]:
scores = cross_validate(hgb, train_input, train_target,
                       return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

0.9321723946453317 0.8801241948619236


In [27]:
# 과대적합을 잘 억제하면서 그레이디언트 부스팅보다 조금 더 높은 성능 제공

In [28]:
# 특성 중요도 확인
# 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지 계산
# 훈련세트, 테스트세트, 추정기 모델에 모두 사용 가능
# n_repeats 랜덤하게 섞을 횟수 지정

In [29]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


HistGradientBoostingClassifier(l2_regularization=0.0, learning_rate=0.1,
                               loss='auto', max_bins=255, max_depth=None,
                               max_iter=100, max_leaf_nodes=31,
                               min_samples_leaf=20, n_iter_no_change=None,
                               random_state=42, scoring=None, tol=1e-07,
                               validation_fraction=0.1, verbose=0,
                               warm_start=False)

In [30]:
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [ ]:
# 평균 출력하면 랜덤 포레스트와 비슷한 비율

In [31]:
# 테스트 세트의 중요도 확인
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [32]:
# 그레이디언트 부스팅과 비슷하게 당도에 집중

In [33]:
# 테스트 세트에서의 성능 최종 확인
hgb.score(test_input, test_target)

0.8723076923076923

In [34]:
# 앙상블 모델은 단일 결정 트리보다 좋은 결과 얻음

## XGBoost
- 사이킷런의 히스토그램 기반 그레이디언트 부스팅 알고리즘 = XGBoost임
- XGBoost는 cross_validate()함수와 함께 사용할 수 있음
- tree_method 매개변수를 hist로 지정하면 히스토그램 기반 그레이디언트 부스팅 사용할 수 있음

In [40]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target,
                       return_train_score=True
                       )

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[17:57:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:57:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:57:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:57:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [41]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


## LightGBM
- 마이크로소프트사에서 만듬
- 빠르고 최신 기술을 많이 적용하고 있어 인기가 높아짐
- 코랩에 바로 테스트 가능
- 사이킷런의 히스토그램 기반 그레이디언트 부스팅이 LightGBM에서 영향을 많이 받음

In [43]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target,
                       return_train_score=True, n_jobs=-1
                       )

In [44]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
